__применение парсера объявлений циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-09-08 14:16:39 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-08 14:16:39 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-08 14:16:41 | AdsListParser: downloader init
[INFO    ] 2022-09-08 14:16:41 | CianParser: downloader init
[INFO    ] 2022-09-08 14:16:41 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-08 14:16:49 | AdsListParser: read page 1
[INFO    ] 2022-09-08 14:17:01 | AdsListParser: read page 2
[INFO    ] 2022-09-08 14:17:11 | AdsListParser: read page 3
[INFO    ] 2022-09-08 14:17:24 | AdsListParser: read page 4
[INFO    ] 2022-09-08 14:17:34 | AdsListParser: read page 5
[INFO    ] 2022-09-08 14:17:48 | AdsListParser: read page 6
[INFO    ] 2022-09-08 14:17:54 | AdsListParser: read page 7
[INFO    ] 2022-09-08 14:18:09 | AdsListParser: read page 8
[INFO    ] 2022-09-08 14:18:15 | AdsListParser: read page 9
[INFO    ] 2022-09-08 14:18:29 | AdsListParser: read page 10
[INFO    ] 2022-09-08 14:18:44 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
298,"3-комн. кв., 74 м², 4/9 этаж",,,10 100 000 ₽,136 486 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. Стрелецкая бухта, улица Вакуленчука, 10]","[2 часа назад, сегодня, 11:30]",https://sevastopol.cian.ru/sale/flat/278015928/,Продаётся трёхкомнатная квартира на ул. Вакуленчука. Цена - 10 100 000,11,2022-09-08 14:25:06.437355
1277,"Студия, 26 м², 1/1 этаж",,,3 450 000 ₽,132 692 ₽/м²,"[Севастополь, р-н Гагаринский, улица Хрусталева, 126]","[вчера, вчера, 17:54]",https://sevastopol.cian.ru/sale/flat/275648553/,"Студия полностью укомплектована с ремонтом , мебелью и техникой.Первый этаж с небольшим своим местом для отдыха.Заходи и живи.",46,2022-09-08 14:25:06.437355
912,"2-комн. кв., 56,4 м², 2/10 этаж",,,7 350 000 ₽,130 319 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. Летчики, проспект Октябрьской Революции, 22/13]","[вчера, вчера, 23:10]",https://sevastopol.cian.ru/sale/flat/270893743/,"Продаю отличную 2-комнатную квартиру 56,4 кв.м. РЯДОМ С МОРЕМ в самом развитом районе на ПОРе!! \nСередина дома, 2 этаж, не угловая. Квартира с удобной планировкой: две просторные изолированные комнаты, раздельный санузел, просторная кухня 9,3 м2, две лоджии. Окна выходят на разные стороны дома: с одной стороны во двор на детскую площадку, а со второй на парковку во дворе. Состояние жилое. При необходимости вся мебель остается. Один собственник, никто не зарегистрирован.\nВ доме в 2021 году сделан капитальный ремонт. Во дворе хорошая детская и спортивная площадки. Возле дома крупный торговый центр Посейдон. \nТихое место, при наличии рядом всей необходимой инфраструктуры: рядом детские сады 131, 111, 125, школы 54 и 58, Билингвальная гимназия 2. Возле дома также находится сквер имени Т.Г. Шевченко, в 500 метрах прекрасный Парк Победы. Пешком до моря 5-10 минут. \nЗвоните! Проведу оперативный показ!\nПомогу с оформлением ипотеки по самым низким ставкам в городе! Полное юридическое сопровождение сделки!\n . Номер в базе: 7391819.",33,2022-09-08 14:25:06.437355


In [7]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1509
1503


In [8]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [ ]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [ ]:
# df

In [ ]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [ ]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [ ]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])